In [7]:
!pip install openai==0.28
!pip install line-bot-sdk
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.1 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, abort
from pyngrok import ngrok, conf
from flask_ngrok import run_with_ngrok
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'

conf.get_default().auth_token = "2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz"

app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

# 質問回数を追跡するための変数
user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    # 文字数チェック
    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            # 回答の文字数をチェックして制限
            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    app.run()


<ipython-input-32-540e4f86fe28>:16: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-32-540e4f86fe28>:17: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://65c8-34-66-199-129.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


<ipython-input-32-540e4f86fe28>:72: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:38:06] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:38:34] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:39:20] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:39:36] "POST / HTTP/1.1" 200 -


In [ ]:
!pip install openai==0.28
!pip install line-bot-sdk
!pip install flask-ngrok
!pip install flask
!pip install pyngrok
!pip install flask pyngrok line-bot-sdk openai

  Using cached line_bot_sdk-3.11.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached aenum-3.1.15-py3-none-any.whl.metadata (3.7 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
Using cached line_bot_sdk-3.11.0-py2.py3-none-any.whl (758 kB)
Using cached aenum-3.1.15-py3-none-any.whl (137 kB)
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached pyngrok-7.2.0-py3-none-any.whl.metadata (7.4 kB)
Using cached pyngrok-7.2.0-py3-none-any.whl (22 kB)


In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'

app = Flask(__name__)

line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

# 質問回数を追跡するための変数
user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    # 文字数チェック
    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            # 回答の文字数をチェックして制限
            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-11-0ad0ed327c51>:11: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-11-0ad0ed327c51>:12: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug

In [ ]:
!git add .
!git commit -m "Fix path issues and update startCommand"
!git push origin main



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git add .
!git commit -m "Update startCommand to use gunicorn"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git add .
!git commit -m "Configure app to use gunicorn and listen on 0.0.0.0:5000"
!git push origin main



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.models import MessageEvent, TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'MessageEvent' from 'linebot.v3.models' (/usr/local/lib/python3.10/dist-packages/linebot/v3/models/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent, TextMessage, TextSendMessage
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'TextMessage' from 'linebot.v3.webhook' (/usr/local/lib/python3.10/dist-packages/linebot/v3/webhook.py)

In [ ]:

import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage, TextSendMessage
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)



ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhooks import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'WebhookHandler' from 'linebot.v3.webhooks' (/usr/local/lib/python3.10/dist-packages/linebot/v3/webhooks/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent  # 修正されたインポートステートメント
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'linebot.v3.messages'

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent  # 修正されたインポートステートメント
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'linebot.v3.messages'

In [ ]:
pip install line-bot-sdk


In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage, TextSendMessage
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))


user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.message import TextMessage, TextSendMessage  # 修正されたインポートステートメント
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'linebot.v3.message'

In [ ]:
!pip install --upgrade line-bot-sdk

In [ ]:
!pip install openai==0.28
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

In [3]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

app = Flask(__name__)

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'
line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-3-759db7b9ed9e>:10: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-3-759db7b9ed9e>:11: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on

In [4]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage, TextSendMessage
from linebot.v3.webhooks import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'
line_bot_api = MessagingApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)



ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [5]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage  # TextSendMessageを削除
from linebot.v3.webhooks import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.models import TextSendMessage  # ここからTextSendMessageをインポート
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))


user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ImportError: cannot import name 'WebhookHandler' from 'linebot.v3.webhooks' (/usr/local/lib/python3.10/dist-packages/linebot/v3/webhooks/__init__.py)

In [6]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage  # TextSendMessageを削除
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import TextSendMessage  # ここからTextSendMessageをインポート
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('OPENAI_API_KEY')
line_bot_api = MessagingApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [8]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))


user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ModuleNotFoundError: No module named 'linebot.v3.messages'

In [9]:
!pip install line-bot-sdk

In [12]:
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))
user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


TypeError: can only concatenate str (not "NoneType") to str

In [4]:
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc")
if not line_channel_access_token:
    raise ValueError("2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz")
if not line_channel_secret:
    raise ValueError("e0069f76b523e833fae449753218f7e1")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ModuleNotFoundError: No module named 'openai'

In [5]:
!pip install line-bot-sdk

In [11]:
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEYが設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKENが設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRETが設定されていません。")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-11-5a0768b4c136>:24: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(line_channel_access_token)
<ipython-input-11-5a0768b4c136>:25: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(line_channel_secret)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [22]:
%env OPENAI_API_KEY=sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc
%env LINE_CHANNEL_ACCESS_TOKEN=2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz
%env LINE_CHANNEL_SECRET=e0069f76b523e833fae449753218f7e1


env: OPENAI_API_KEY=sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc
env: LINE_CHANNEL_ACCESS_TOKEN=2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz
env: LINE_CHANNEL_SECRET=e0069f76b523e833fae449753218f7e1


In [12]:
!pip install line-bot-sdk==3.0.0


ERROR: Could not find a version that satisfies the requirement line-bot-sdk==3.0.0 (from versions: 1.0.0, 1.0.1, 1.0.2, 1.1.0, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.9.0, 1.10.0, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.14.0, 1.15.0, 1.16.0, 1.17.0, 1.18.0, 1.19.0, 1.20.0, 2.0.1, 2.1.0, 2.2.1, 2.3.0, 2.4.1, 2.4.2, 2.4.3, 3.0.2, 3.0.3, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.5.1, 3.6.0, 3.7.0, 3.8.0, 3.8.1, 3.9.0, 3.10.1, 3.10.2, 3.11.0)
ERROR: No matching distribution found for line-bot-sdk==3.0.0


In [13]:
!pip install line-bot-sdk


In [14]:
import os
from flask import Flask, request, abort
from linebot.v3 import MessagingApi, WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEYが設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKENが設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRETが設定されていません。")

openai.api_key = openai_api_key
line_bot_api = MessagingApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ImportError: cannot import name 'MessagingApi' from 'linebot.v3' (/usr/local/lib/python3.10/dist-packages/linebot/v3/__init__.py)

In [15]:
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEYが設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKENが設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRETが設定されていません。")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-15-5a0768b4c136>:24: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(line_channel_access_token)
<ipython-input-15-5a0768b4c136>:25: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(line_channel_secret)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [16]:
!pip install gunicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 876.1 kB/s eta 0:00:00


In [18]:
!python app.py

python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [20]:
!gunicorn app:app --bind 0.0.0.0:5000

[2024-08-04 05:25:35 +0000] [5425] [INFO] Starting gunicorn 22.0.0
[2024-08-04 05:25:35 +0000] [5425] [INFO] Listening at: http://0.0.0.0:5000 (5425)
[2024-08-04 05:25:35 +0000] [5425] [INFO] Using worker: sync
[2024-08-04 05:25:35 +0000] [5426] [INFO] Booting worker with pid: 5426
[2024-08-04 05:25:35 +0000] [5426] [ERROR] Exception in worker process
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/arbiter.py", line 609, in spawn_worker
    worker.init_process()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/workers/base.py", line 134, in init_process
    self.load_wsgi()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/workers/base.py", line 146, in load_wsgi
    self.wsgi = self.app.wsgi()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/app/base.py", line 67, in wsgi
    self.callable = self.load()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/app/wsgiapp.py", line 58, in load
    return self.load_wsgia

In [21]:
!pip install flask line-bot-sdk openai


In [23]:
%env OPENAI_API_KEY=sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc
%env LINE_CHANNEL_ACCESS_TOKEN=2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz
%env LINE_CHANNEL_SECRET=e0069f76b523e833fae449753218f7e1


env: OPENAI_API_KEY=sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc
env: LINE_CHANNEL_ACCESS_TOKEN=2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz
env: LINE_CHANNEL_SECRET=e0069f76b523e833fae449753218f7e1


In [25]:
!gunicorn app:app --bind 0.0.0.0:5000


[2024-08-04 05:38:41 +0000] [8565] [INFO] Starting gunicorn 22.0.0
[2024-08-04 05:38:41 +0000] [8565] [INFO] Listening at: http://0.0.0.0:5000 (8565)
[2024-08-04 05:38:41 +0000] [8565] [INFO] Using worker: sync
[2024-08-04 05:38:41 +0000] [8566] [INFO] Booting worker with pid: 8566
[2024-08-04 05:38:41 +0000] [8566] [ERROR] Exception in worker process
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/arbiter.py", line 609, in spawn_worker
    worker.init_process()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/workers/base.py", line 134, in init_process
    self.load_wsgi()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/workers/base.py", line 146, in load_wsgi
    self.wsgi = self.app.wsgi()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/app/base.py", line 67, in wsgi
    self.callable = self.load()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/app/wsgiapp.py", line 58, in load
    return self.load_wsgia

In [27]:
!ngrok http 5000

http - start an HTTP tunnel

USAGE:
  ngrok http [address:port | port] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080                        # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --domain mydomain.com                           # run ngrok with your own custom domain
  ngrok http 80 --allow-cidr 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your a

In [29]:
!ngrok config add-authtoken 2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [1]:
!ngrok http 5000


Traceback (most recent call last):
  File "/usr/lib/python3.10/subprocess.py", line 1209, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1959, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.10/subprocess.py", line 1917, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py", line 582, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py", line 570, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 276, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.10/subprocess.py", line 347, in call
    return p.wait(ti

In [2]:
!pip install pyngrok


In [4]:
!python run_ngrok.py

python3: can't open file '/content/run_ngrok.py': [Errno 2] No such file or directory


In [6]:
!pip install pyngrok flask line-bot-sdk openai


In [8]:
!python run_ngrok.py


python3: can't open file '/content/run_ngrok.py': [Errno 2] No such file or directory


In [9]:
cd /Users/sasakihitomi/line_chatbot0803/run_ngrok.py

[Errno 2] No such file or directory: '/Users/sasakihitomi/line_chatbot0803/run_ngrok.py'
/content


In [10]:
cd run_ngrok.py

[Errno 2] No such file or directory: 'run_ngrok.py'
/content


In [11]:
cd /Users/sasakihitomi/line_chatbot0803/run_ngrok.py


[Errno 2] No such file or directory: '/Users/sasakihitomi/line_chatbot0803/run_ngrok.py'
/content


In [13]:
!python run_ngrok.py


python3: can't open file '/content/run_ngrok.py': [Errno 2] No such file or directory


In [14]:
cd /Users/sasakihitomi/line_chatbot0803/run_ngrok.py


[Errno 2] No such file or directory: '/Users/sasakihitomi/line_chatbot0803/run_ngrok.py'
/content


In [15]:
pip install pyngrok flask line-bot-sdk openai


In [17]:
!python run_ngrok.py


python3: can't open file '/content/run_ngrok.py': [Errno 2] No such file or directory


In [18]:
pwd


'/content'

In [19]:
ls


sample_data/


In [23]:
!mkdir line_chatbot0803
!cd line_chatbot0803


mkdir: cannot create directory ‘line_chatbot0803’: File exists


In [24]:
cd Users

[Errno 2] No such file or directory: 'Users'
/content


In [25]:
cd sasakihitomi

[Errno 2] No such file or directory: 'sasakihitomi'
/content


In [26]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    print(f"Received message from {user_id}: {user_message}")  # 追加

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0]['message']['content']
            print(f"Generated reply: {reply_message}")  # 追加

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )


NameError: name 'handler' is not defined

In [27]:
from pyngrok import ngrok
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY が設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKEN が設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRET が設定されていません。")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    print(f"Received message from {user_id}: {user_message}")

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']
            print(f"Generated reply: {reply_message}")

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    # `ngrok`トンネルを開く
    public_url = ngrok.connect(5000)
    print(f"ngrok public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


ValueError: OPENAI_API_KEY が設定されていません。

In [28]:
from pyngrok import ngrok
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY が設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKEN が設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRET が設定されていません。")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    print(f"Received message from {user_id}: {user_message}")

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ]
            )
            reply_message = response.choices[0].message['content']
            print(f"Generated reply: {reply_message}")

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    # `ngrok`トンネルを開く
    public_url = ngrok.connect(5000)
    print(f"ngrok public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


ValueError: OPENAI_API_KEY が設定されていません。

In [29]:
from pyngrok import ngrok
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY が設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKEN が設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRET が設定されていません。")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    print(f"Received message from {user_id}: {user_message}")

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ]
            )
            reply_message = response.choices[0].message['content']
            print(f"Generated reply: {reply_message}")

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    # `ngrok`トンネルを開く
    public_url = ngrok.connect(5000)
    print(f"ngrok public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


ValueError: OPENAI_API_KEY が設定されていません。

In [30]:
pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [31]:
%env OPENAI_API_KEY=sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc
%env LINE_CHANNEL_ACCESS_TOKEN=2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz
%env LINE_CHANNEL_SECRET=e0069f76b523e833fae449753218f7e1


env: OPENAI_API_KEY=sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc
env: LINE_CHANNEL_ACCESS_TOKEN=2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz
env: LINE_CHANNEL_SECRET=e0069f76b523e833fae449753218f7e1


In [32]:
from pyngrok import ngrok
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai_api_key = os.getenv('OPENAI_API_KEY')
line_channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.getenv('LINE_CHANNEL_SECRET')

# 環境変数が取得できているかチェック
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY が設定されていません。")
if not line_channel_access_token:
    raise ValueError("LINE_CHANNEL_ACCESS_TOKEN が設定されていません。")
if not line_channel_secret:
    raise ValueError("LINE_CHANNEL_SECRET が設定されていません。")

openai.api_key = openai_api_key
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    print(f"Received message from {user_id}: {user_message}")

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": system_instruction},
                        {"role": "user", "content": user_message}
                    ]
                )
                reply_message = response.choices[0].message['content']
                print(f"Generated reply: {reply_message}")

                if len(reply_message) > 250:
                    reply_message = reply_message[:250] + '...'
            except Exception as e:
                print(f"Error generating reply: {e}")
                reply_message = "エラーが発生しました。再度お試しください。"

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    try:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=reply_message)
        )
    except Exception as e:
        print(f"Error sending reply: {e}")

if __name__ == "__main__":
    # `ngrok`トンネルを開く
    public_url = ngrok.connect(5000)
    print(f"ngrok public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


<ipython-input-32-68d6360bf353>:25: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(line_channel_access_token)
<ipython-input-32-68d6360bf353>:26: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(line_channel_secret)


ngrok public URL: NgrokTunnel: "https://7c4e-34-68-41-145.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
